# Imports

In [1]:
!pip install lime
!pip install keras_preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 KB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283859 sha256=fc90e831b3c272994ee825479a658b80efdb4d47f0bb59b685771ca8a153001f
  Stored in directory: /root/.cache/pip/wheels/ed/d7/c9/5a0130d06d6310bc6cbe55220e6e72dcb8c4eff9a478717066
Successfully built lime
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 2.7 MB/s eta 0:00:00


In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve
from sklearn.preprocessing import StandardScaler,OrdinalEncoder

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
df=pd.read_csv("/content/drive/My Drive/ISM/IoTID20.csv")
print("Number of Columns:", len(df.columns))
print("Number of Samples:",len(df))

Number of Columns: 86
Number of Samples: 625783


# Data Preprocessing

In [5]:
categories=df['Cat'].unique()
df.drop(df[df['Flow_Byts/s']==float('inf')].index,inplace=True)

drop=['Fwd_PSH_Flags','Fwd_URG_Flags','Fwd_Byts/b_Avg','Fwd_Pkts/b_Avg','Fwd_Blk_Rate_Avg','Bwd_Byts/b_Avg','Bwd_Pkts/b_Avg','Bwd_Blk_Rate_Avg','Init_Fwd_Win_Byts','Fwd_Seg_Size_Min','Cat','Label']

df.drop(drop,axis=1,inplace=True)

from datetime import datetime
def get_timestamp(str):
  date_format = '%d/%m/%Y %I:%M:%S %p'
  timestamp = datetime.strptime(str, date_format).time()
  timestamp = float(timestamp.hour) + float(timestamp.minute)/60 + float(timestamp.second)/3600
  return timestamp
df['Timestamp']=df['Timestamp'].apply(lambda str:get_timestamp(str))

In [6]:
print("Number of Columns: ", len(df.columns))
print(df.columns)

Number of Columns:  74
Index(['Flow_ID', 'Src_IP', 'Src_Port', 'Dst_IP', 'Dst_Port', 'Protocol',
       'Timestamp', 'Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts',
       'TotLen_Fwd_Pkts', 'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Max',
       'Fwd_Pkt_Len_Min', 'Fwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std',
       'Bwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Std', 'Flow_Byts/s', 'Flow_Pkts/s', 'Flow_IAT_Mean',
       'Flow_IAT_Std', 'Flow_IAT_Max', 'Flow_IAT_Min', 'Fwd_IAT_Tot',
       'Fwd_IAT_Mean', 'Fwd_IAT_Std', 'Fwd_IAT_Max', 'Fwd_IAT_Min',
       'Bwd_IAT_Tot', 'Bwd_IAT_Mean', 'Bwd_IAT_Std', 'Bwd_IAT_Max',
       'Bwd_IAT_Min', 'Bwd_PSH_Flags', 'Bwd_URG_Flags', 'Fwd_Header_Len',
       'Bwd_Header_Len', 'Fwd_Pkts/s', 'Bwd_Pkts/s', 'Pkt_Len_Min',
       'Pkt_Len_Max', 'Pkt_Len_Mean', 'Pkt_Len_Std', 'Pkt_Len_Var',
       'FIN_Flag_Cnt', 'SYN_Flag_Cnt', 'RST_Flag_Cnt', 'PSH_Flag_Cnt',
       'ACK_Flag_Cnt', 'URG_Flag_Cnt', 'CWE_Flag_Count', 'ECE_Flag_Cnt',
       

In [7]:
df.head()

,Flow_ID,Src_IP,Src_Port,Dst_IP,Dst_Port,Protocol,Timestamp,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,...,Fwd_Act_Data_Pkts,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Sub_Cat
0,192.168.0.13-192.168.0.16-10000-10101-17,192.168.0.13,10000,192.168.0.16,10101,17,3.431389,75,1,1,...,1,0.0,0.0,0.0,0.0,75.0,0.000000,75.0,75.0,Mirai-Ackflooding
1,192.168.0.13-222.160.179.132-554-2179-6,222.160.179.132,2179,192.168.0.13,554,6,22.185000,5310,1,2,...,0,0.0,0.0,0.0,0.0,2655.0,2261.327486,4254.0,1056.0,DoS-Synflooding
2,192.168.0.13-192.168.0.16-9020-52727-6,192.168.0.16,52727,192.168.0.13,9020,6,1.413333,141,0,3,...,0,0.0,0.0,0.0,0.0,70.5,0.707107,71.0,70.0,Scan Port OS
3,192.168.0.13-192.168.0.16-9020-52964-6,192.168.0.16,52964,192.168.0.13,9020,6,3.971389,151,0,2,...,0,0.0,0.0,0.0,0.0,151.0,0.000000,151.0,151.0,Mirai-Hostbruteforceg
4,192.168.0.1-239.255.255.250-36763-1900-17,192.168.0.1,36763,239.255.255.250,1900,17,1.688333,153,2,1,...,2,0.0,0.0,0.0,0.0,76.5,0.707107,77.0,76.0,Mirai-Hostbruteforceg


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 625415 entries, 0 to 625782
Data columns (total 74 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow_ID            625415 non-null  object 
 1   Src_IP             625415 non-null  object 
 2   Src_Port           625415 non-null  int64  
 3   Dst_IP             625415 non-null  object 
 4   Dst_Port           625415 non-null  int64  
 5   Protocol           625415 non-null  int64  
 6   Timestamp          625415 non-null  float64
 7   Flow_Duration      625415 non-null  int64  
 8   Tot_Fwd_Pkts       625415 non-null  int64  
 9   Tot_Bwd_Pkts       625415 non-null  int64  
 10  TotLen_Fwd_Pkts    625415 non-null  float64
 11  TotLen_Bwd_Pkts    625415 non-null  float64
 12  Fwd_Pkt_Len_Max    625415 non-null  float64
 13  Fwd_Pkt_Len_Min    625415 non-null  float64
 14  Fwd_Pkt_Len_Mean   625415 non-null  float64
 15  Fwd_Pkt_Len_Std    625415 non-null  float64
 16  Bw

# Data preperation

In [9]:
cat_atr=['Flow_ID', 'Src_IP','Src_Port','Dst_IP','Dst_Port','Protocol',
         'Bwd_PSH_Flags','Bwd_URG_Flags','FIN_Flag_Cnt','SYN_Flag_Cnt',
         'RST_Flag_Cnt','PSH_Flag_Cnt','ACK_Flag_Cnt','URG_Flag_Cnt',
         'CWE_Flag_Count','ECE_Flag_Cnt']
num_atr=[]
for x in df.columns:
  if x in ['Sub_Cat']:
    continue
  if x not in cat_atr:
    num_atr.append(x)

In [14]:
X= df.drop(['Sub_Cat'],axis=1)
y= df['Sub_Cat']


X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_test,X_validation,y_test,y_validation=train_test_split(X_test,y_test,test_size=0.5,random_state=42)

print("Training Samples:",len(X_train))
print("Testing Samples:",len(X_test))
print("Validation Samples:",len(X_validation))

Training Samples: 500332
Testing Samples: 62541
Validation Samples: 62542


In [15]:
def prepare_input_data(df,cat_atr,encoder):
  
  scaler=StandardScaler()

  cat_data=encoder.transform(df[cat_atr])
  cat_data=pd.DataFrame(cat_data,columns=cat_atr)
  cat_data=np.asarray(cat_data)
  
  num_data=df.drop(cat_atr,axis=1)
  num_data=np.asarray(num_data).astype(np.float32)
  num_data = scaler.fit_transform(num_data)
  
  return (num_data,cat_data)

def prepare_labels(data,target_cols):
  labels=np.zeros((len(data),9))
  i=0
  for x in data:
    labels[i][np.where(target_cols==x)[0][0]]=1
    i+=1
  return labels


In [16]:
encoder=OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)
encoder.fit(df[cat_atr])
(X_train_num,X_train_cat)=prepare_input_data(X_train,cat_atr,encoder)
(X_validation_num,X_validation_cat)=prepare_input_data(X_validation,cat_atr,encoder)
(X_test_num,X_test_cat)=prepare_input_data(X_test,cat_atr,encoder)

In [17]:
target_cols=df['Sub_Cat'].unique()
y_train=prepare_labels(y_train,target_cols)
y_validation=prepare_labels(y_validation,target_cols)
y_test=prepare_labels(y_test,target_cols)

In [18]:
n=0
for x in encoder.categories_:
  n+=len(x)

# Model

In [27]:
def create_model():
  #cat input
  inp_cat=tf.keras.layers.Input(shape=np.shape(X_train_cat)[1],name="categorical_input")
  embedded = tf.keras.layers.Embedding(input_dim=n, output_dim=32, input_length=np.shape(X_train_cat)[1],name="categorical_embedding")(inp_cat)
  flattened = tf.keras.layers.Flatten(name="flatten")(embedded)

  # num input
  inp_num=tf.keras.layers.Input(shape=len(num_atr),name="numerical_input")

  concatenated = tf.keras.layers.Concatenate(name="concatenation")([inp_num, flattened])

  # hidden layers
  hidden= keras.layers.Dense(569, activation='selu',kernel_initializer="lecun_normal")(concatenated)
  hidden= keras.layers.Dense(569, activation='selu',kernel_initializer="lecun_normal")(hidden)
  hidden= keras.layers.Dense(569, activation='selu',kernel_initializer="lecun_normal")(hidden)
  hidden= keras.layers.Dense(569, activation='selu',kernel_initializer="lecun_normal")(hidden)
  hidden= keras.layers.Dense(569, activation='selu',kernel_initializer="lecun_normal")(hidden)
  hidden= keras.layers.Dense(569, activation='selu',kernel_initializer="lecun_normal")(hidden)
  hidden= keras.layers.Dense(569, activation='selu',kernel_initializer="lecun_normal")(hidden)
  hidden= keras.layers.Dense(569, activation='selu',kernel_initializer="lecun_normal")(hidden)
  hidden= keras.layers.Dense(569, activation='selu',kernel_initializer="lecun_normal")(hidden)

  # output
  output= keras.layers.Dense(9, activation='softmax',name="Output")(hidden)

  model = keras.Model(
        inputs=[inp_cat,inp_num],
        outputs=[output],
    )
  loss="categorical_crossentropy"
  optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-4)
  metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
  model.compile(loss=loss, optimizer=optimizer,metrics=metrics)
  return model

In [24]:
# Create distribution strategy
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
strategy = tf.distribute.TPUStrategy(tpu)

In [28]:
use_tpu = True
if use_tpu: 
    # Create model
    with strategy.scope():
        model = create_model()
else:
  model = create_model()

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 categorical_input (InputLayer)  [(None, 16)]        0           []                               
                                                                                                  
 categorical_embedding (Embeddi  (None, 16, 32)      4313248     ['categorical_input[0][0]']      
 ng)                                                                                              
                                                                                                  
 numerical_input (InputLayer)   [(None, 57)]         0           []                               
                                                                                                  
 flatten (Flatten)              (None, 512)          0           ['categorical_embedding[0][

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)
checkpoint_cb = keras.callbacks.ModelCheckpoint('/content/drive/My Drive/ISM/classifier.h5',save_best_only=True)
tr_history=model.fit([X_train_cat,X_train_num],y_train,validation_data=([X_validation_cat,X_validation_num],y_validation),
          epochs=50,callbacks=[early_stopping_cb,checkpoint_cb])

Epoch 1/50
15636/15636 [==============================] - 410s 25ms/step - loss: 0.3787 - accuracy: 0.7974 - precision_2: 0.9308 - recall_2: 0.7236 - val_loss: 0.3367 - val_accuracy: 0.8089 - val_precision_2: 0.9719 - val_recall_2: 0.7238
Epoch 2/50
15636/15636 [==============================] - 380s 24ms/step - loss: 0.3396 - accuracy: 0.8081 - precision_2: 0.9663 - recall_2: 0.7246 - val_loss: 0.3346 - val_accuracy: 0.8094 - val_precision_2: 0.9777 - val_recall_2: 0.7235
Epoch 3/50
15636/15636 [==============================] - 383s 25ms/step - loss: 0.3341 - accuracy: 0.8095 - precision_2: 0.9744 - recall_2: 0.7237 - val_loss: 0.3317 - val_accuracy: 0.8106 - val_precision_2: 0.9794 - val_recall_2: 0.7234
Epoch 4/50
15636/15636 [==============================] - 376s 24ms/step - loss: 0.3303 - accuracy: 0.8099 - precision_2: 0.9784 - recall_2: 0.7229 - val_loss: 0.3272 - val_accuracy: 0.8108 - val_precision_2: 0.9792 - val_recall_2: 0.7238
Epoch 5/50
15636/15636 [====================

In [ ]:
model_evaluation=model.evaluate([X_test_cat,X_test_num],y_test)

In [ ]:
import statistics
print("Loss: ", model_evaluation[0])
print("Accuracy Score: ", model_evaluation[1])
print("Precision Score: ", model_evaluation[2])
print("Recall Score: ", model_evaluation[3])
print("F1 Score: ", statistics.harmonic_mean([model_evaluation[2],model_evaluation[3]]))

# Explanations

In [ ]:
# model=create_model()
# model.load_weights('/content/drive/My Drive/ISM/classifier.h5')

# model = keras.models.load_model("/content/drive/My Drive/ISM/classifier.h5")
